In [44]:
import numpy as np
import awkward as ak
import uproot_methods

In [45]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

In [46]:
def stack_arrays(a, keys, axis=-1):
    flat_arr = np.stack([a[k].flatten() for k in keys], axis=axis)
    return awkward.JaggedArray.fromcounts(a[keys[0]].counts, flat_arr)

In [47]:
def pad_array(a, maxlen, value=0., dtype='float32'):
    x = (np.ones((len(a), maxlen)) * value).astype(dtype)
    for idx, s in enumerate(a):
        if not len(s):
            continue
        trunc = s[:maxlen].astype(dtype)
        x[idx, :len(trunc)] = trunc
    return x

In [48]:
##and Professor suggests that we could use mass, classifacation for later application
def SetAKArr(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    n_particles_ls = []
    px_ls = []
    py_ls = []
    pz_ls = []
    energy_ls = []
    mass_ls = []
    charge_ls = []
    _label1 = []
    _label2 = []
    _label3 = []
    _label4 = []
    _label5 = []
    
    n = 0
    for line in lines:
        if line.startswith('E'):
            if not n == 0:
                n_particles_ls.append(n)
                n = 0
            exp_inf = line.split()
#             _label1.append(int(exp_inf[1]))
#             _label2.append(1-int(exp_inf[1]))
#             _label1.append(1)
#             _label2.append(0)
            _label1.append(float(exp_inf[1]))
            _label2.append(float(exp_inf[2]))
            _label3.append(float(exp_inf[3]))
            _label4.append(float(exp_inf[4]))
            _label5.append(float(exp_inf[5]))
        else:
            par = line.split()
            ##particle +1
            n = n + 1
            px_ls.append(abs(float(par[2])))
            py_ls.append(abs(float(par[3])))
            pz_ls.append(abs(float(par[4])))
            energy_ls.append(abs(float(par[5])))
            mass_ls.append(abs(float(par[6])))  
            charge_ls.append(int(par[0]))
#             px_ls.append(6)
#             py_ls.append(2)
#             pz_ls.append(3)
#             energy_ls.append(4)
#             mass_ls.append(5)
    if not n == 0:
        n_particles_ls.append(n)
    px_arr = np.array(px_ls)
    py_arr = np.array(py_ls)
    pz_arr = np.array(pz_ls)
    energy_arr = np.array(energy_ls)
    mass_arr = np.array(mass_ls)
    charge_arr = np.array(charge_ls)
    n_particles = np.array(n_particles_ls)

#     print(n_particles)
    px = ak.JaggedArray.fromcounts(n_particles, px_arr)
    py = ak.JaggedArray.fromcounts(n_particles, py_arr)
    pz = ak.JaggedArray.fromcounts(n_particles, pz_arr)
    energy = ak.JaggedArray.fromcounts(n_particles, energy_arr)
    mass = ak.JaggedArray.fromcounts(n_particles, mass_arr)
    charge = ak.JaggedArray.fromcounts(n_particles, charge_arr)
    p4 = uproot_methods.TLorentzVectorArray.from_cartesian(px, py, pz, energy)
    ##Create an Order Dic
    from collections import OrderedDict
    v = OrderedDict()
    v['part_px'] = px
#     print(px)
    v['part_py'] = py
    v['part_pz'] = pz
    v['part_energy'] = energy
    v['part_mass'] = mass
    v['charge'] = charge
    v['part_e_log'] = np.log(energy)
    v['part_px_log'] = np.log(px)
    v['part_py_log'] = np.log(py)
    v['part_pz_log'] = np.log(pz)
    v['part_m_log'] = np.log(mass)
#     ls1 = [1,2,3,4]
#     ls2 = [5,6,7,8]
#     v['label'] = np.stack((_label1, _label2, _label3, _label4, _label5), axis=-1)
#     print(v['label'])
#     v['label'] = np.stack((_label1, _label2, _label3, _label4, _label5), axis = -1)
    v['label'] = np.stack(_label5, axis = -1)
#     print(v['label'])
    return v

In [49]:
class Dataset(object):
    def __init__(self, filepath, feature_dict = {}, label = 'label', pad_len=100, data_format='channel_first'):
        self.filepath = filepath
        self.feature_dict = feature_dict
        if len(feature_dict) == 0:
            feature_dict['points'] = ['part_px','part_py','part_pz']
            feature_dict['features'] = ['part_energy', 'part_mass', 'charge', 'part_px', 'part_py', 'part_pz']
            feature_dict['mask'] = ['part_energy']
        ##currently we use 'E' for experiments
        self.label = label
        self.pad_len = pad_len
        assert data_format in ('channel_first', 'channel_last')
        self.stack_axis = 1 if data_format=='channel_first' else -1
        self._values = {}
        self._label = None
        self._load()
        
    def _load(self):
        logging.info('Start loading file %s' % self.filepath)
#         counts = None
        a = SetAKArr(self.filepath)
        self._label = a[self.label]
        for k in self.feature_dict:
                cols = self.feature_dict[k]
                if not isinstance(cols, (list, tuple)):
                    cols = [cols]
                arrs = []
                for col in cols:
                    arrs.append(pad_array(a[col], self.pad_len))
                    ##check the dimesion of a[col], and it should be array.
                self._values[k] = np.stack(arrs, axis=self.stack_axis)
        logging.info('Finished loading file %s' % self.filepath)
        
        
        
    def __len__(self):
        return len(self._label)

    def __getitem__(self, key):
        if key==self.label:
            return self._label
        else:
            return self._values[key]
    
    @property
    def X(self):
        return self._values
    
    @property
    def y(self):
        return self._label

    def shuffle(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        shuffle_indices = np.arange(self.__len__())
        np.random.shuffle(shuffle_indices)
        for k in self._values:
            self._values[k] = self._values[k][shuffle_indices]
        self._label = self._label[shuffle_indices]

In [50]:
train_dataset = Dataset('train.txt', data_format='channel_last')
val_dataset = Dataset('val.txt', data_format='channel_last')
test_dataset = Dataset('test.txt', data_format = 'channel_last')

[2024-03-03 10:51:47,299] INFO: Start loading file train.txt
/home/htk/miniforge3/envs/tensorflow/lib/python3.5/site-packages/awkward/array/jagged.py:976: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[2024-03-03 10:51:47,482] INFO: Finished loading file train.txt
[2024-03-03 10:51:47,483] INFO: Start loading file val.txt
[2024-03-03 10:51:47,506] INFO: Finished loading file val.txt
[2024-03-03 10:51:47,507] INFO: Start loading file test.txt
[2024-03-03 10:51:47,529] INFO: Finished loading file test.txt


In [51]:
import tensorflow as tf
from tensorflow import keras
from tf_keras_model import get_particle_net, get_particle_net_lite

In [52]:
model_type = 'particle_net_lite' # choose between 'particle_net' and 'particle_net_lite'
##this shows the number of classes for classification
# num_classes = train_dataset.y.shape[1]
num_classes = 1
# print(num_classes)
input_shapes = {k:train_dataset[k].shape[1:] for k in train_dataset.X}
if 'lite' in model_type:
    model = get_particle_net_lite(num_classes, input_shapes)
else:
    model = get_particle_net(num_classes, input_shapes)

In [53]:
# Training parameters
batch_size = 1024 if 'lite' in model_type else 384
epochs = 30

In [54]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    logging.info('Learning rate: %f'%lr)
    return lr

In [55]:
# model.compile(loss='categorical_crossentropy',
#               optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
#               metrics=['accuracy'])
model.compile(loss='log_cosh',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
# model.compile(loss='mean_squared_error',
#               optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
#               metrics=['accuracy'])
model.summary()

[2024-03-03 10:51:53,232] INFO: Learning rate: 0.001000


Model: "ParticleNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mask (InputLayer)               [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
tf_op_layer_NotEqual_1 (TensorF [(None, 100, 1)]     0           mask[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_Cast_2 (TensorFlowO [(None, 100, 1)]     0           tf_op_layer_NotEqual_1[0][0]     
__________________________________________________________________________________________________
tf_op_layer_Equal_1 (TensorFlow [(None, 100, 1)]     0           tf_op_layer_Cast_2[0][0]         
________________________________________________________________________________________

In [56]:
from tensorflow.keras.callbacks import Callback
class LossLogger(Callback):
    def __init__(self, filename):
        super().__init__()
        self.filename = filename

    def on_epoch_end(self, epoch, logs=None):
        with open(self.filename, 'a') as f:
#             print("Epoch ", epoch + 1,": loss = ", logs["val_loss"], "\n", file = f)
#             if (epoch+1)%5==0 or epoch==0:
            print(logs["val_loss"], file = f)
#             f.write()
loss_logger = LossLogger('loss_log.txt')

In [57]:
# Prepare model model saving directory.
import os
save_dir = 'model_checkpoints'
model_name = '%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
# checkpoint = keras.callbacks.ModelCheckpoint(filepath='loss.txt',
#                              monitor='val_acc',
#                              verbose=1,
#                              save_best_only=True)
#I change the monitor from val_acc to val_loss
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)
lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [checkpoint, lr_scheduler, loss_logger]
# callbacks = [lr_schedule]

In [58]:
train_dataset.shuffle()
model.fit(train_dataset.X, train_dataset.y,
          batch_size=batch_size,
#           epochs=epochs,
          epochs=30, # --- train only for 1 epoch here for demonstration ---
          validation_data=(val_dataset.X, val_dataset.y),
          shuffle=True,
          callbacks=callbacks)

[2024-03-03 10:51:56,584] INFO: Learning rate: 0.001000


Epoch 1/30
5/5 [==============================] - ETA: 0s - accuracy: 0.0000e+00 - loss: 4.2870
Epoch 00001: val_loss improved from inf to 4.48587, saving model to model_checkpoints/particle_net_lite_model.001.h5
5/5 [==============================] - 3s 649ms/step - accuracy: 0.0000e+00 - loss: 4.2870 - val_accuracy: 0.0000e+00 - val_loss: 4.4859


[2024-03-03 10:52:01,506] INFO: Learning rate: 0.001000


Epoch 2/30
5/5 [==============================] - ETA: 0s - accuracy: 0.0000e+00 - loss: 3.4625
Epoch 00002: val_loss improved from 4.48587 to 4.37961, saving model to model_checkpoints/particle_net_lite_model.002.h5
5/5 [==============================] - 3s 622ms/step - accuracy: 0.0000e+00 - loss: 3.4625 - val_accuracy: 0.0000e+00 - val_loss: 4.3796


[2024-03-03 10:52:05,516] INFO: Learning rate: 0.001000


...

Epoch 28/30
5/5 [==============================] - ETA: 0s - accuracy: 0.0000e+00 - loss: 0.0559
Epoch 00028: val_loss improved from 3.15302 to 3.11386, saving model to model_checkpoints/particle_net_lite_model.028.h5
5/5 [==============================] - 3s 649ms/step - accuracy: 0.0000e+00 - loss: 0.0559 - val_accuracy: 0.0000e+00 - val_loss: 3.1139


[2024-03-03 10:53:53,591] INFO: Learning rate: 0.000100


Epoch 29/30
5/5 [==============================] - ETA: 0s - accuracy: 0.0000e+00 - loss: 0.0568
Epoch 00029: val_loss improved from 3.11386 to 3.07454, saving model to model_checkpoints/particle_net_lite_model.029.h5
5/5 [==============================] - 3s 664ms/step - accuracy: 0.0000e+00 - loss: 0.0568 - val_accuracy: 0.0000e+00 - val_loss: 3.0745


[2024-03-03 10:53:57,856] INFO: Learning rate: 0.000100


Epoch 30/30
5/5 [==============================] - ETA: 0s - accuracy: 0.0000e+00 - loss: 0.0542
Epoch 00030: val_loss improved from 3.07454 to 3.03444, saving model to model_checkpoints/particle_net_lite_model.030.h5
5/5 [==============================] - 3s 654ms/step - accuracy: 0.0000e+00 - loss: 0.0542 - val_accuracy: 0.0000e+00 - val_loss: 3.0344


In [43]:
predictions = model.predict(test_dataset.X)
for prediction in predictions:
    print(prediction)

[2.2355587]
[2.263892]
[1.9904692]
[2.0716598]
[2.1739566]
[2.1478882]
[2.0731783]
[2.1058273]
[2.1574616]
[2.061681]
[2.2142441]
[2.153094]
[2.0303738]
[2.0304387]
[2.1040561]
[2.1510427]
[2.1811533]
[2.0896368]
[2.156884]
[2.0285914]
[2.0999713]
[2.1493244]
[2.1204877]
[2.005537]
[2.1498725]
[2.2014647]
[2.1142707]
[2.141198]
[2.2180314]
[2.1213305]
[2.1032257]
[2.0453799]
[2.101873]
[2.1368816]
[2.1217015]
[2.2747025]
[2.1699953]
[2.1023476]
[2.1776586]
[2.0868165]
[2.1228402]
[2.0104]
[2.169025]
[2.2086189]
[2.0947545]
[2.1251929]
[2.0240119]
[2.205694]
[2.1174688]
[2.2067692]
[2.0292854]
[2.1025047]
[2.106222]
[2.0701768]
[2.1069267]
[2.2311242]
[2.2329934]
[2.1579502]
[2.1621525]
[2.1256273]
[1.9916127]
[1.989718]
[2.0969675]
[2.0354187]
[2.061754]
[2.016888]
[2.2054718]
[2.1494567]
[2.0751963]
[2.2164342]
[2.0571394]
[2.092072]
[2.1033242]
[2.117317]
[2.2183459]
[2.1128912]
[2.1693864]
[2.2150085]
[2.2009718]
[2.1880624]
[2.216729]
[2.1379833]
[2.075752]
[2.1628091]
[2.144723]
[